We need to fix the waveform generation so that it isn't just 1 or 0 for a pixel. We need the waveform to be set up for temporal accuracy

In [89]:
from PIL import Image
import os
import numpy as np
import nifgen

In [149]:
#image = r"C:\Users\max\Desktop\NanoStride\skull_slices\skull00011.png"
image_path = r"C:\Users\max\Desktop\NanoStride\skull_slices"
sampling_rate = 10_000_000
waveforms = []

for i in os.listdir(image_path):
    image = os.path.join(image_path, i)
    # Calculate number of pixels in the first row and save the variable
    img = Image.open(image).convert('L')
    binary = (np.array(img) > 128).astype(np.uint8)
    pixels_per_row = binary.shape[1]

    # Calculate amount of time for each pixel
    scan_frequency = 7911.78
    time_per_row = (1/scan_frequency)/2
    time_per_pixel = time_per_row/pixels_per_row
    datapoints_per_pixel = np.floor(time_per_pixel * sampling_rate)

    # Create black, white, gap array
    black = np.ones(int(datapoints_per_pixel))*(-1)
    white = np.ones(int(datapoints_per_pixel))
    gap = np.ones(int(datapoints_per_pixel * pixels_per_row))*-1

    for row_index in range(pixels_per_row):
        row = binary[row_index]
        for column in row:
            if column == 0:
                waveforms.append(black)
            else:
                waveforms.append(white)
        waveforms.append(gap)

waveform_list = np.concatenate(waveforms)
waveform_numpy = np.array(waveform_list).astype(np.float64)

In [150]:
import os
import numpy as np
from PIL import Image

# Path to image directory
image_path = r"C:\Users\max\Desktop\NanoStride\skull_slices"

# Constants
sampling_rate = 10_000_000  # 10 MS/s
scan_frequency = 7911.48    # Resonant mirror frequency (Hz)
waveforms = []

# Time per scanline (bottom half of cycle)
time_per_row = 1 / (2 * scan_frequency)  # seconds per row (~63.2 µs)
samples_per_row = int(round(time_per_row * sampling_rate))  # e.g. 632 samples

print(f"Samples per row (half-cycle): {samples_per_row}")

# Process each image
for filename in os.listdir(image_path):
    image = os.path.join(image_path, filename)
    img = Image.open(image).convert('L')
    binary = (np.array(img) > 128).astype(np.uint8)
    
    n_rows = binary.shape[0]
    pixels_per_row = binary.shape[1]

    base_samples_per_pixel = samples_per_row // pixels_per_row
    remainder = samples_per_row % pixels_per_row

    print(f"{filename}: {pixels_per_row} pixels → {base_samples_per_pixel} samples/pixel + {remainder} extra")

    # Create the waveform row by row
    for row in binary:
        for i, pixel in enumerate(row):
            extra = 1 if i < remainder else 0
            n_samples = base_samples_per_pixel + extra
            value = 1.0 if pixel else -1.0
            waveforms.append(np.ones(n_samples, dtype=np.float64) * value)

        # Add a "gap" for the top half of the mirror swing (same length as row)
        waveforms.append(np.ones(samples_per_row, dtype=np.float64) * -1.0)

# Final waveform
waveform_numpy = np.concatenate(waveforms).astype(np.float64)


Samples per row (half-cycle): 632
skull00000.png: 300 pixels → 2 samples/pixel + 32 extra
skull00001.png: 300 pixels → 2 samples/pixel + 32 extra
skull00002.png: 300 pixels → 2 samples/pixel + 32 extra
skull00003.png: 300 pixels → 2 samples/pixel + 32 extra
skull00004.png: 300 pixels → 2 samples/pixel + 32 extra
skull00005.png: 300 pixels → 2 samples/pixel + 32 extra
skull00006.png: 300 pixels → 2 samples/pixel + 32 extra
skull00007.png: 300 pixels → 2 samples/pixel + 32 extra
skull00008.png: 300 pixels → 2 samples/pixel + 32 extra
skull00009.png: 300 pixels → 2 samples/pixel + 32 extra
skull00010.png: 300 pixels → 2 samples/pixel + 32 extra
skull00011.png: 300 pixels → 2 samples/pixel + 32 extra
skull00012.png: 300 pixels → 2 samples/pixel + 32 extra
skull00013.png: 300 pixels → 2 samples/pixel + 32 extra
skull00014.png: 300 pixels → 2 samples/pixel + 32 extra
skull00015.png: 300 pixels → 2 samples/pixel + 32 extra
skull00016.png: 300 pixels → 2 samples/pixel + 32 extra
skull00017.png

In [151]:
waveform_numpy.nbytes

452006400

In [152]:
# Max allocation space in bytes
max_mem = 80 * 1024 * 1024  # 160 MiB = 167_772_160 bytes
target_chunk = max_mem // 10  # Ideal chunk size ~16.7 MiB

# Find best chunk size that evenly divides max_mem
for chunk_size in range(target_chunk, 0, -1):
    if max_mem % chunk_size == 0:
        break  # Found the best chunk size

# Now compute how many elements that is
bytes_per_element = waveform_numpy.itemsize
elements_per_chunk = chunk_size // bytes_per_element

print(f"Max mem: {max_mem} bytes")
print(f"Chunk size: {chunk_size} bytes")
print(f"Elements per chunk: {elements_per_chunk}")
print(f"Total chunks needed: {int(np.ceil(waveform_numpy.nbytes / chunk_size))}")

# Create the chunked list of numpy arrays
waveform_chunks = [
    waveform_numpy[i : i + elements_per_chunk]
    for i in range(0, len(waveform_numpy), elements_per_chunk)
]

print(f"Generated {len(waveform_chunks)} chunks.")

Max mem: 83886080 bytes
Chunk size: 8388608 bytes
Elements per chunk: 1048576
Total chunks needed: 54
Generated 54 chunks.


In [158]:
import time
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 10_000_000

    #80 MiB allocation
    waveform_handle = session.allocate_waveform(max_mem)
    session.streaming_waveform_handle = waveform_handle

    # Write 10, 16MB chunks
    for i in range(10):
        session.write_waveform(waveform_handle, waveform_chunks[i])

    # Configure trigger
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.trigger_mode = nifgen.TriggerMode.SINGLE

    # Initiate
    session.initiate()

    # Write the rest of the chunks
    for i in range(10, len(waveform_chunks)):
        print(f'Writing Chunk {i}')
        session.write_waveform(waveform_handle, waveform_chunks[i])

Writing Chunk 10
Writing Chunk 11
Writing Chunk 12
Writing Chunk 13
Writing Chunk 14
Writing Chunk 15
Writing Chunk 16
Writing Chunk 17
Writing Chunk 18
Writing Chunk 19
Writing Chunk 20
Writing Chunk 21
Writing Chunk 22
Writing Chunk 23
Writing Chunk 24
Writing Chunk 25
Writing Chunk 26
Writing Chunk 27
Writing Chunk 28
Writing Chunk 29
Writing Chunk 30
Writing Chunk 31
Writing Chunk 32
Writing Chunk 33
Writing Chunk 34
Writing Chunk 35
Writing Chunk 36
Writing Chunk 37
Writing Chunk 38
Writing Chunk 39
Writing Chunk 40
Writing Chunk 41
Writing Chunk 42
Writing Chunk 43
Writing Chunk 44
Writing Chunk 45
Writing Chunk 46
Writing Chunk 47
Writing Chunk 48
Writing Chunk 49
Writing Chunk 50
Writing Chunk 51
Writing Chunk 52
Writing Chunk 53


In [64]:
max_mem / chunk_size

10.0

In [60]:
83886080 / chunk_size

5.0

In [62]:
83886080 / 1024 / 1024

80.0

In [ ]:
def create_bidirectional_waveforms(image_dir, serpentine=True):
    waveforms = []

    image_files = sorted([
        f for f in os.listdir(image_dir)
        if f.endswith(".png")
    ])

    for i, filename in enumerate(image_files):
        path = os.path.join(image_dir, filename)
        img = Image.open(path).convert("L")
        binary = (np.array(img) > 128).astype(np.uint8)
        for row_index in range(binary.shape[0]):
            row = binary[row_index]
            for column in row:
                if column == 0:
                    waveforms.append(black)
                else:
                    waveforms.append(white)
        return np.concatenate(waveforms)

In [16]:
import numpy as np
import os
from PIL import Image # Pillow library for image processing

def create_bidirectional_waveforms(
    image_dir: str,
    serpentine: bool = True,
    samples_per_channel_increment: int = 64
) -> list[np.ndarray]:
    """
    Generates a waveform from a sequence of binary images, outputting it
    as a list of 16 MB NumPy array chunks of float64 data. Each chunk's
    sample count and the total waveform's sample count will be an integer
    multiple of samples_per_channel_increment.

    Args:
        image_dir (str): The directory containing the PNG image files.
        serpentine (bool): If True, odd-numbered rows will be read in reverse
                           order (left-to-right, then right-to-left, etc.).
        samples_per_channel_increment (int): The required sample increment for
                                             hardware compatibility (e.g., 64).
                                             The total samples and each chunk's
                                             samples will be a multiple of this.

    Returns:
        list[np.ndarray]: A list of NumPy arrays, where each array is a chunk
                          of the generated waveform with dtype float64.
                          Each chunk will be approximately 16 MB and its sample
                          count will be a multiple of the increment.
                          Returns an empty list if no images are found or an
                          error occurs.
    """
    # Define black and white levels for float64 output, normalized between -1 and 1
    black_level = -1.0
    white_level = 1.0

    all_samples = []

    try:
        image_files = sorted([
            f for f in os.listdir(image_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")) # Support common image formats
        ])

        if not image_files:
            print(f"No image files found in directory: {image_dir}")
            return []

        print(f"Processing {len(image_files)} image(s) from {image_dir}...")

        for i, filename in enumerate(image_files):
            path = os.path.join(image_dir, filename)
            try:
                # Open image, convert to grayscale, then to binary (0 or 255)
                img = Image.open(path).convert("L")
                binary_array = (np.array(img) > 128).astype(np.uint8) # Convert to 0 or 1

                for row_index in range(binary_array.shape[0]):
                    row = binary_array[row_index]
                    # Apply serpentine logic if enabled for odd rows
                    if serpentine and row_index % 2 == 1: # Odd rows (0-indexed)
                        row = row[::-1] # Reverse row for serpentine scanning

                    for column_value in row:
                        if column_value == 0: # Black pixel
                            all_samples.append(black_level)
                        else: # White pixel
                            all_samples.append(white_level)
            except Exception as e:
                print(f"Error processing image {filename}: {e}")
                continue # Skip to next image

    except FileNotFoundError:
        print(f"Error: Directory not found: {image_dir}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred during image loading: {e}")
        return []

    if not all_samples:
        print("No samples generated from images. Returning empty list.")
        return []

    # Convert the list of samples to a NumPy array of float64
    full_waveform = np.array(all_samples, dtype=np.float64)

    # --- Ensure total samples are a multiple of samples_per_channel_increment ---
    num_samples_raw = full_waveform.shape[0]
    num_samples = (num_samples_raw + samples_per_channel_increment - 1) // samples_per_channel_increment * samples_per_channel_increment

    # Pad the waveform with the black_level if necessary to meet the aligned total samples
    if num_samples > num_samples_raw:
        padding_needed = num_samples - num_samples_raw
        full_waveform = np.pad(full_waveform, (0, padding_needed), 'constant', constant_values=black_level)
        print(f"Padded waveform from {num_samples_raw} to {num_samples} samples for alignment.")

    print(f"Total samples generated for waveform: {num_samples}")

    # --- Chunking logic (16MB chunks of float64) ---
    chunk_size_mb = 16
    bytes_per_sample = 8 # float64 is 8 bytes
    chunk_size_bytes = chunk_size_mb * 1024 * 1024
    chunk_size_samples_raw = chunk_size_bytes // bytes_per_sample

    # Ensure the desired chunk size for each segment is a multiple of the increment
    aligned_chunk_size_samples = (chunk_size_samples_raw + samples_per_channel_increment - 1) // samples_per_channel_increment * samples_per_channel_increment

    waveform_chunks = []
    start_index = 0
    while start_index < num_samples:
        end_index = min(start_index + aligned_chunk_size_samples, num_samples)
        current_chunk = full_waveform[start_index:end_index]

        # This check should ideally pass if num_samples and aligned_chunk_size_samples are correct
        if current_chunk.shape[0] % samples_per_channel_increment != 0:
            print(f"Error: A chunk of size {current_chunk.shape[0]} was generated which is not a multiple of the increment {samples_per_channel_increment}. This should not happen with current logic.")

        waveform_chunks.append(current_chunk)
        start_index = end_index

    print(f"Waveform chunked into {len(waveform_chunks)} segments.")
    return waveform_chunks

# Example Usage (requires a directory named 'images' with PNG files)
# import os
# # Create a dummy image directory and files for testing if they don't exist
# if not os.path.exists("test_images"):
#     os.makedirs("test_images")
#     # Create a simple 10x10 black and white image
#     dummy_img_data = np.random.randint(0, 2, size=(10, 10), dtype=np.uint8) * 255
#     dummy_img = Image.fromarray(dummy_img_data, 'L')
#     dummy_img.save("test_images/dummy_image_01.png")
#     dummy_img.save("test_images/dummy_image_02.png")
#     print("Created dummy images in 'test_images' directory for demonstration.")

waveform_chunks_float64 = create_bidirectional_waveforms(image_dir=r"C:\Users\max\Desktop\NanoStride\skull_slices", serpentine=True)
if waveform_chunks_float64:
    print(f"Generated {len(waveform_chunks_float64)} chunks from images.")
    for i, chunk in enumerate(waveform_chunks_float64[:3]):
        print(f"  Chunk {i+1} shape: {chunk.shape}, Dtype: {chunk.dtype}, Size (MB): {chunk.nbytes / (1024*1024):.2f}")
        print(f"  Chunk {i+1} samples % {64} = {chunk.shape[0] % 64}")

Processing 149 image(s) from C:\Users\max\Desktop\NanoStride\skull_slices...
Total samples generated for waveform: 149000000
Waveform chunked into 72 segments.
Generated 72 chunks from images.
  Chunk 1 shape: (2097152,), Dtype: float64, Size (MB): 16.00
  Chunk 1 samples % 64 = 0
  Chunk 2 shape: (2097152,), Dtype: float64, Size (MB): 16.00
  Chunk 2 samples % 64 = 0
  Chunk 3 shape: (2097152,), Dtype: float64, Size (MB): 16.00
  Chunk 3 samples % 64 = 0


In [ ]:
# Need to save as a binary waveform now
waveform_normalized = correct_waveform.astype(np.float64) * 2 -1
#waveform_normalized.astype('<f8').tofile(f"correct_waveform.bin")

In [17]:
chunk_size = 16 * 1024 * 1024
import time
with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 10_000_000
    #160 MB allocation
    waveform_handle = session.allocate_waveform(83886080)
    session.streaming_waveform_handle = waveform_handle

    print(session.streaming_space_available_in_waveform)

    # Write 10, 16MB chunks
    for i in range(10):
        session.write_waveform(waveform_handle, waveform_chunks_float64[i])
    # Initiate
    session.initiate()

    print(session.streaming_space_available_in_waveform)

    # Write the rest of the chunks
    for i in range(10, len(waveform_chunks_float64)):
        print(f'Writing Chunk {i}')
        print(session.streaming_space_available_in_waveform)
        session.write_waveform(waveform_handle, waveform_chunks_float64[i])
        print(session.streaming_space_available_in_waveform)




83886080
62917440
Writing Chunk 10
62921792
61514688
Writing Chunk 11
61517120
60050944
Writing Chunk 12
60053312
58586368
Writing Chunk 13
58588800
57123712
Writing Chunk 14
57127744
55664704
Writing Chunk 15
55667200
54201792
Writing Chunk 16
54204160
52737152
Writing Chunk 17
52739456
51272832
Writing Chunk 18
51276544
49810944
Writing Chunk 19
49813248
48347392
Writing Chunk 20
48349696
46883712
Writing Chunk 21
46886080
45419392
Writing Chunk 22
45422208
43948352
Writing Chunk 23
43950720
42484352
Writing Chunk 24
42486720
41022784
Writing Chunk 25
41025152
39558464
Writing Chunk 26
39566016
38101632
Writing Chunk 27
38104192
36638080
Writing Chunk 28
36640384
35173696
Writing Chunk 29
35176064
33709952
Writing Chunk 30
33718336
32252032
Writing Chunk 31
32254464
30788480
Writing Chunk 32
30790848
29324544
Writing Chunk 33
29326912
27861184
Writing Chunk 34
27865152
26403072
Writing Chunk 35
26405632
24934976
Writing Chunk 36
24937472
23470912
Writing Chunk 37
23473472
22007040
Wr

Now each pixel has the correct timing. However, it isn't synchronized for the scanning. We need to have it synchronized to the mirror scanner.

This means that each row needs a gap.

In [1]:
import nifgen
import time

with nifgen.Session("Dev1") as session:
    session.output_mode = nifgen.OutputMode.ARB
    session.arb_sample_rate = 100_000_000
    wf_handle = session.create_waveform(waveform_normalized)
    session.start_trigger_type = nifgen.StartTriggerType.DIGITAL_EDGE
    session.digital_edge_start_trigger_edge = nifgen.StartTriggerDigitalEdgeEdge.RISING
    session.trigger_mode = nifgen.TriggerMode.CONTINUOUS
    session.digital_edge_start_trigger_source = "/Dev1/PFI1"
    session.configure_arb_waveform(wf_handle, gain=1, offset=0)
    session.initiate()
    time.sleep(5)

NameError: name 'waveform_normalized' is not defined